In [ ]:
import numpy as np #Processamiento numérico
import scipy as sc #Amplia numpy
import matplotlib.pyplot as plt #Para dibujar
import time 

#Los datos para el primer ejemplo
from sklearn.datasets import make_circles #Dataset

#Los datos para los números
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', cache=False)

In [ ]:
#CLASSE DE LA CAPA DE LA RED

class neural_layer():
  #Especificar número de conexiones. Núméro de neuronas. Función de activación.
  def __init__(self,n_conn, n_neur,act_f):
    self.act_f=act_f
    self.b = np.random.rand(1, n_neur)      *2-1 #Vector columna
    self.W = np.random.rand(n_conn, n_neur) *2-1 #Para que el random pase de 0,1 a -1,1

In [ ]:
#FUNCIONES DE ACTIVACIÓN

#lambda para indicar función anónima
sign = (lambda x: 1 / (1 + np.e**(-x)),
        lambda x: x*(1-x))

_x = np.linspace(-5,5,100) #100 índices entre -5,5 para dibujar la función.
plt.plot(_x,sign[0](_x)) #El índice nos permite elegir si entrar a la derivada o a la función original.

In [ ]:
#FUNCIÓN ENCARGADA DE CREAR LA RED NEURONAL
 
def create_nn(topology, act_f):
  nn = []
  for i,layer in enumerate(topology[:-1]): #Para evitar overflow.
    nn.append(neural_layer(topology[i],topology[i+1],act_f))
  return nn


In [ ]:
#FUNCIÓN ENCARGADA DE ENTRENAR LA RED NEURONAL
l2_cost = (lambda Yp,Yr : np.mean((Yp-Yr)**2), #Error cuadrático medio.
           lambda Yp,Yr : (Yp-Yr)) #La derivada del error.

#X datos de entrada; Y datos de salid; Definidos al inicio.
def train(neural_net, X, Y, cost, lr=0.5, train = True): #0.5 learning rate que nos permite saber en que grado actualizamos. Demasiado grande puede hacer saltos. Pequeño que sea demasiado lento.
  out = [(None,X)] #Guardaremos tanto el valor de la suma ponderada como el de activación.
  
  #Forward pass: ir pasando el vector de entrada capa por capa.
  for l,layer in enumerate(neural_net):


    z = out[-1][1] @ neural_net[l].W + neural_net[l].b #Producto matricial.
    a = neural_net[0].act_f[0](z)
 
    out.append((z,a))
  
  if train:
    #Backward pass
    deltas = [] 

    for i in reversed(range(0, len(neural_net))):

      z = out[i+1][0]
      a = out[i+1][1]
      
      #Miramos si estamos en la última capa
      if (i==len(neural_net)-1):
        deltas.insert(0,cost[1](a,Y)*neural_net[i].act_f[1](a))
      else:
        deltas.insert(0, deltas[0] @ _W.T * neural_net[i].act_f[1](a))

      #Guardamos temporalmente la W que será la siguiente en la próxima iteración.
      _W = neural_net[i].W

      #Gradient descent || NOS ESTAMOS REFERIENDO A LOS ÍNDICES DE LA CAPA ANTERIOR
      neural_net[i].b = neural_net[i].b - np.mean(deltas[0], axis = 0, keepdims = True)*lr
      neural_net[i].W = neural_net[i].W - out[i][1].T@deltas[0] *lr 

  return out[-1][1]


In [ ]:
#EJEMPLO 2D sencillo

n = 500 #Número de registros en la base de datos.
p = 2 #Características sobre cada dato.
o = 1 #Número de salidas

#Creamos el dataset
X,Y=make_circles(n_samples=n,factor=0.5, noise=0.05) #Factor distancia entre circulos.
Y = Y[:,np.newaxis]
#X nos indica posición.
#Y nos indica a que set de datos pertenece. En este caso es true i false para indicar el circulo exterior o interior.

plt.scatter(X[Y[:,0]==0,0],X[Y[:,0]==0,1], c= "skyblue")
plt.scatter(X[Y[:,0]==1,0],X[Y[:,0]==1,1], c="salmon")
plt.axis("equal") #Para que sea escala cuadrada
plt.show()
print("X: ",X.shape, "Y: ",Y.shape)

In [ ]:
from IPython.display import clear_output

topology = [p,8,4,1] #Última capa una neurona de salida. Elegido de forma arbitraria el número.

neural_n = create_nn(topology, sign)
loss = []

for i in range(1000):
  #Entrenamos la red
  pY = train(neural_n,X,Y,l2_cost,0.05)

  #Printeamos cada 100 iteraciones
  if i%100 == 0:
    loss.append(l2_cost[0](pY,Y))
    res = 50
    _x0 = np.linspace(-1.5,1.5,res)
    _x1 = np.linspace(-1.5,1.5,res)

    _Yshow = np.zeros((res,res))

    for i0,x0 in enumerate(_x0):
      for i1,x1 in enumerate (_x1):
        _Yshow[i0,i1] = train(neural_n,np.array([[x0,x1]]),Y, l2_cost,train=False)[0][0]
    
    plt.pcolormesh(_x0,_x1,_Yshow,cmap="coolwarm")
    plt.axis("equal")

    plt.scatter(X[Y[:,0]==0,0],X[Y[:,0]==0,1], c= "skyblue")
    plt.scatter(X[Y[:,0]==1,0],X[Y[:,0]==1,1], c= "salmon")

    clear_output(wait=True)
    plt.show()
    plt.plot(range(len(loss)),loss)
    plt.show()
    time.sleep(0.5)

print(loss)


In [ ]:
#EJEMPLO NÚMEROS

n = 5000 #Número de registros en la base de datos.
p = 784 #Características sobre cada dato: corresponden a 28*28 píxeles
o = 1 #Número de salidas

#Esta función nos sirve para imprimir el número de la base de datos por pantalla.
def dibuja(item):
  res = 28
  _x0 = np.linspace(0,1,res)
  #La y está invertida, porque
  _x1 = np.linspace(1,0,res)

  plt.pcolormesh(_x0,_x1,item.reshape(res, res),cmap="Greys")
  plt.axis("equal")
  plt.show()

#preprocesamos los datos de entrada para el entrenamiento de la red neuronal. Vamos a processar imágenes de 28*28 píxeles.
_X = mnist.data.astype('float32')
#los valores de cada pixel deben estar entre 0 y 1
_X /= 255.0

#para la clasificación, cada vector de salida tendrá 10 números, una para cada dígito:
#esto se llama one hot encoding
_Y_int = mnist.target.astype('int64') #Esto són lo números.
Y = _Y_int[:n]
X = _X[:n,:] #Cogemos solo los que usaremos para entrenar
#Vamos a coger los diferentes Y para cada red neuronal.

Y1 = np.copy(Y)
Y1 = Y1[:,np.newaxis]
for i, value  in enumerate(Y1):
  if value[0] <= 4:
    Y1[i][0] = 0.0
  else:
    Y1[i][0] = 1.0

Y2 = np.copy(Y)
Y2 = Y2[:,np.newaxis]
for i, value  in enumerate(Y2):
  if value[0] % 2 == 0:
    Y2[i][0] = 0.0
  else:
    Y2[i][0] = 1.0

Y3 = np.copy(Y)
Y3 = Y3[:,np.newaxis]
for i, value  in enumerate(Y3):
  if value[0] in (2,3,6,7,9):
    Y3[i][0] = 0.0
  else:
    Y3[i][0] = 1.0

Y4 = np.copy(Y)
Y4 = Y4[:,np.newaxis]
for i, value  in enumerate(Y4):
  if value[0] in (2,3,4,8,9):
    Y4[i][0] = 0.0
  else:
    Y4[i][0] = 1.0

In [ ]:
#Comprobamos que esten bien los conjuntos

check = []
values = [0,0,0,0,0,0,0,0,0,0]
for i in range(100):
  if Y[i] not in check:
    check.append(Y[i])
    values[Y[i]]=i

for i in range(10):
  j = values[i]
  print("REAL: ", Y[j], "A les diferents llistes: ",Y1[j]," ",Y2[j]," ",Y3[j]," ",Y4[j])

#La Lista 1 o Y1 es la primera columna del print etc.

In [ ]:
#RED1
print("RED1")
c1=196
c2=40
reps = 5001

topology1 = [p,c1,c2,o] #Última capa una neurona de salida. Elegido de forma arbitraria el número.
 
neural_n1 = create_nn(topology1, sign)
loss = []
timeA = time.time()
for i in range(reps):
  #Entrenamos la red
  pY = train(neural_n1,X,Y1,l2_cost,0.001)
  #Printeamos cada 100 iteraciones
  if i%100 == 0:
    loss.append(l2_cost[0](pY,Y1))
    print("Iteración número ",i," valor ",loss[-1], " y ha trigat ", time.time() - timeA)
    timeA = time.time()

#RED2
print("RED2")

topology2 = [p,c1,c2,o] #Última capa una neurona de salida. Elegido de forma arbitraria el número.
 
neural_n2 = create_nn(topology2, sign)
loss = []
timeA = time.time()
for i in range(reps):
  #Entrenamos la red
  pY = train(neural_n2,X,Y2,l2_cost,0.001)
  #Printeamos cada 100 iteraciones
  if i%100 == 0:
    loss.append(l2_cost[0](pY,Y2))
    print("Iteración número ",i," valor ",loss[-1], " y ha trigat ", time.time() - timeA)
    timeA = time.time()

#RED3
print("RED3")

topology3 = [p,c1,c2,o] #Última capa una neurona de salida. Elegido de forma arbitraria el número.
 
neural_n3 = create_nn(topology3, sign)
loss = []
timeA = time.time()
for i in range(reps):
  #Entrenamos la red
  pY = train(neural_n3,X,Y3,l2_cost,0.001)
  #Printeamos cada 100 iteraciones
  if i%100 == 0:
    loss.append(l2_cost[0](pY,Y3))
    print("Iteración número ",i," valor ",loss[-1], " y ha trigat ", time.time() - timeA)
    timeA = time.time()

#RED4
print("RED4")

topology4 = [p,c1,c2,o] #Última capa una neurona de salida. Elegido de forma arbitraria el número.
 
neural_n4 = create_nn(topology4, sign)
loss = []
timeA = time.time()
for i in range(reps):
  #Entrenamos la red
  pY = train(neural_n4,X,Y4,l2_cost,0.001)
  #Printeamos cada 100 iteraciones
  if i%100 == 0:
    loss.append(l2_cost[0](pY,Y4))
    print("Iteración número ",i," valor ",loss[-1], " y ha trigat ", time.time() - timeA)
    timeA = time.time()
print(loss)
 

In [ ]:
#Usa las 4 redes neuronales para saber que número és
def WhatNum(nnet1,nnet2,nnet3,nnet4,image):
  prob = [1.,1.,1.,1.,1.,1.,1.,1.,1.,1.]
  set1 = train(nnet1,image,Y1,l2_cost,0.001,False)[0][0]
  set2 = train(nnet2,image,Y2,l2_cost,0.001,False)[0][0]
  set3 = train(nnet3,image,Y3,l2_cost,0.001,False)[0][0]
  set4 = train(nnet4,image,Y4,l2_cost,0.001,False)[0][0]
  if set1>=0.5:
    set1=0.9
  else:
   set1 = 0.1
  if set2>=0.5:
    set2=0.9
  else:
   set2 = 0.1
  if set3>=0.5:
    set3=0.9
  else:
   set3 = 0.1
  if set4>=0.5:
    set4=0.9
  else:
   set4 = 0.1


  for i in range(10):
    if i>4:
      prob[i]=prob[i]*set1
    else:
      prob[i]=prob[i]*(1-set1)
    if i%2==0:
      prob[i]=prob[i]*(1-set2)
    else: 
      prob[i]=prob[i]*set2
    if i in (1,4,5,8,0):
      prob[i]=prob[i]*set3
    else:
      prob[i]=prob[i]*(1-set3)
    if i in (0,1,5,6,7):
      prob[i]=prob[i]*set4
    else:
      prob[i]=prob[i]*(1-set4)


  numberP = -10
  suposedN = 11

  for i in range(10):
    if(prob[i]>numberP):
      numberP = prob[i]
      suposedN = i
  return suposedN




In [ ]:
asserts = 0
for i in range(10000):
  position = 5000+i
  num = _X[position]
  numS = num[:,np.newaxis]
  numS = numS.T
  suposed = WhatNum(neural_n1,neural_n2,neural_n3,neural_n4,numS)
  real = _Y_int[position]
  if(suposed==real):
    asserts+=1

print("Assert rate: ", asserts/10000)

In [ ]:
position = 1009
num = _X[position]
numS = num[:,np.newaxis]
numS = numS.T
print(train(neural_n1,numS,Y1,l2_cost,0.001,False)[0][0])
dibuja(numS[0])